In [3]:
import os

print("原始工作路径:", os.getcwd())
os.chdir("/workspace/")
print("新工作路径:", os.getcwd())

原始工作路径: /workspace
新工作路径: /workspace


In [4]:
!apt-get -y update
!apt-get -y upgrade
!apt-get install -y cmake
!apt-get install -y libcurl4-gnutls-dev

Get:1 http://mirrors.cloud.tencent.com/ubuntu focal InRelease [265 kB]
Hit:2 http://mirrors.cloud.tencent.com/ubuntu focal-security InRelease         
Hit:3 http://mirrors.cloud.tencent.com/ubuntu focal-updates InRelease          
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Fetched 265 kB in 1s (463 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Calculating upgrade... Done
The following packages have been kept back:
  libcudnn9-cuda-12 libcudnn9-dev-cuda-12 libnccl-dev libnccl2
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
cmake is already the newest version (3.16.3-1ubuntu1.20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.
Reading package lists... Done


In [5]:
%pip install --upgrade pip
%pip install --upgrade ipywidgets

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [6]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-14B-Instruct-bnb-4bit",
    load_in_4bit = True,
    use_gradient_checkpointing = True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.3: Fast Qwen2 patching. Transformers: 4.57.1.
   \\   /|    Tesla V100-SXM2-32GB. Num GPUs = 1. Max memory: 31.739 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
model = FastLanguageModel.get_peft_model(
   model,
   r = 16, 
   target_modules = ["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj", "down_proj",],
   lora_alpha = 16,
   lora_dropout = 0, 
   bias = "none",    
   use_gradient_checkpointing = "unsloth", 
   random_state = 3407,
   use_rslora = False, 
   loftq_config = None, 
)

Unsloth 2025.11.3 patched 48 layers with 48 QKV layers, 48 O layers and 48 MLP layers.


In [8]:
from datasets import load_dataset
dataset = load_dataset('json', data_files='med-train-dateset.json', split='train')
chat_template = """下面是一些描述一些任务的说明。编写适当完成每个请求的响应。

### 指令:
{INPUT}

### 回应:
{OUTPUT}"""

from unsloth import apply_chat_template

dataset = apply_chat_template(
    dataset,
    tokenizer=tokenizer,
    chat_template=chat_template,
   
)
print(dataset[0])

Generating train split: 0 examples [00:00, ? examples/s]

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/119397 [00:00<?, ? examples/s]

{'conversations': [{'content': '怀孕37周我憋的上宝宝胎动多胎心异常怎么回事', 'role': 'user'}, {'content': '胎动频繁，胎心快，要考虑胎儿是不是缺氧的表现。建议去医院做胎心监护，如果评分低于9分，要吸氧等处理。', 'role': 'assistant'}], 'text': '下面是一些描述一些任务的说明。编写适当完成每个请求的响应。\n\n### 指令:\n怀孕37周我憋的上宝宝胎动多胎心异常怎么回事\n\n### 回应:\n胎动频繁，胎心快，要考虑胎儿是不是缺氧的表现。建议去医院做胎心监护，如果评分低于9分，要吸氧等处理。<|im_end|>'}


In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False, 
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", 
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=12):   0%|          | 0/119397 [00:00<?, ? examples/s]

[accelerate.utils.other|WARNING]Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [10]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 119,397 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 68,812,800 of 14,838,846,464 (0.46% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.355300
2,3.859300
3,3.700000
4,3.089900
5,3.037700
6,2.975000
7,3.011800
8,2.610900
9,2.357000
10,2.298500


In [11]:
FastLanguageModel.for_inference(model) 
messages = [                   
    {"role": "user", "content": "我肚子疼"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


可能是肠痉挛引起的，建议服用山莨菪碱片治疗<|im_end|>


In [12]:
model.save_pretrained_merged("qwen25-14b-offical-finetuned-bnb-4bit", tokenizer, save_method="merged_4bit_forced",)

Unsloth: Merging LoRA weights into 4bit model...


/root/miniforge3/lib/python3.11/site-packages/peft/tuners/lora/bnb.py:397: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Unsloth: Merging finished.
Unsloth: Found skipped modules: ['lm_head']. Updating config.
Unsloth: Saving merged 4bit model to qwen25-14b-offical-finetuned-bnb-4bit...
Unsloth: Merged 4bit model saved.
Unsloth: Merged 4bit model process completed.
